In [2]:
import os
import csv
import pandas as pd
import numpy as np

import datetime
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import squarify

from sklearn import model_selection, preprocessing, metrics
plt.style.use('fivethirtyeight')

print(os.getcwd())
print(os.listdir("../"))

ModuleNotFoundError: No module named 'plotly'

In [8]:
df = pd.read_csv("../mag_2018.csv")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1211000 entries, 0 to 1210999
Data columns (total 16 columns):
Unnamed: 0     1211000 non-null int64
AA             1211000 non-null object
C              50789 non-null object
CC             1211000 non-null int64
D              1211000 non-null object
ECC            1211000 non-null int64
F              1197152 non-null object
Id             1211000 non-null int64
J              1019164 non-null object
RId            835626 non-null object
Ti             1211000 non-null object
VFN            1147111 non-null object
W              1210997 non-null object
Y              1211000 non-null int64
kesEntityId    1211000 non-null int64
logprob        1211000 non-null float64
dtypes: float64(1), int64(6), object(9)
memory usage: 147.8+ MB


In [13]:
def get_block_avg_usage(df):
    for dtype in ['float','int','object']:
        selected_dtype = df.select_dtypes(include=[dtype])
        mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
        mean_usage_mb = mean_usage_b / 1024 ** 2
        print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))
        del mean_usage_b, mean_usage_mb, selected_dtype

In [133]:
df.head(5)

,Authors,Conference,Citation_count,Date,Expected_count,Domain,Id,Journal,References,Title,Venue,Words,Year,Popularity,Month
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,-16.064,1
1,"[{'S': 1, 'AuId': 2618360078, 'AfN': 'technica...",NaN,1787,2018-02-23,1787,"[{'FId': 2779853757, 'FN': 'penicillic acid'},...",2789134836,"{'JN': 'world mycotoxin journal', 'JId': 70096...","[2073569767, 1989318771, 2109782899, 214495155...",a critical review of producers of small lacton...,World Mycotoxin Journal,"['acid', 'critical', 'lactone', 'moniliformin'...",2018,-16.192,2
2,"[{'AuId': 2498139363, 'S': 1, 'AuN': 'r r m pa...",NaN,1785,2018-03-01,1785,"[{'FId': 2779550045, 'FN': 'patulin'}, {'FId':...",2786183769,"{'JN': 'int j food microbiol', 'JId': 132626406}",NaN,alternative patulin pathway unproven,International Journal of Food Microbiology,"['alternative', 'pathway', 'patulin', 'unproven']",2018,-16.193,3
3,"[{'S': 1, 'AuId': 2127898042, 'AfN': 'google',...",NaN,1957,2018-04-01,1957,"[{'FId': 65885262, 'FN': 'scale space segmenta...",2412782625,"{'JN': 'pami', 'JId': 199944782}","[1686810756, 2618530766, 2194775991, 209711776...",deeplab semantic image segmentation with deep ...,IEEE Transactions on Pattern Analysis and Mach...,"['atrous', 'connected', 'convolution', 'convol...",2018,-16.388,4
4,"[{'S': 1, 'AuId': 2761591088, 'AfN': 'leiden u...",NaN,293,2018-08-01,293,"[{'FId': 2778316820, 'FN': 'photometric system...",2798336535,"{'JN': 'astron astrophys', 'JId': 205231332}","[2011301426, 2142220522, 2061939373, 205293750...",gaia data release 2 summary of the contents an...,Astronomy and Astrophysics,"['2', 'contents', 'data', 'gaia', 'properties'...",2018,-16.455,8


In [127]:
# Make more sensible column names
rename_dict = {"AA": "Authors", 
               "C": "Conference", 
               "CC": "Citation_count",
               "ECC": "Expected_count",
               "D": "Date", 
               "F": "Domain", 
               "J": "Journal", 
               "RId": "References", 
               "Ti": "Title", 
               "VFN": "Venue", 
               "W": "Words", 
               "Y": "Year", 
               "logprob":"Popularity",
               "Unnamed: 0": "Row_num"}
df.rename(index=str, columns=rename_dict, inplace=True)

In [92]:
df.select_dtypes(include="number").describe()

,Row_num,Citation_count,ECC,Id,Year,kesEntityId,Popularity
count,1.211000e+06,1.211000e+06,1.211000e+06,1.211000e+06,1211000.0,1.211000e+06,1.211000e+06
mean,6.054995e+05,1.064620e+00,1.064997e+00,2.797143e+09,2018.0,2.260905e+08,-2.118847e+01
std,3.495857e+05,1.755177e+01,1.755261e+01,1.486480e+08,0.0,9.040128e+07,6.017534e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,4.727120e+05,2018.0,1.190005e+06,-2.200200e+01
25%,3.027498e+05,0.000000e+00,0.000000e+00,2.785024e+09,2018.0,1.142518e+08,-2.167500e+01
50%,6.054995e+05,0.000000e+00,0.000000e+00,2.799726e+09,2018.0,2.779343e+08,-2.129900e+01
75%,9.082492e+05,1.000000e+00,1.000000e+00,2.883550e+09,2018.0,2.864866e+08,-2.081000e+01
max,1.210999e+06,6.651000e+03,6.651000e+03,2.898614e+09,2018.0,2.941568e+08,-1.606400e+01


In [94]:
df.select_dtypes(exclude="number").describe()

,Authors,Conference,Date,Domain,Journal,References,Title,Venue,Words
count,1211000,50789,1211000,1197152,1019164,835626,1211000,1147111,1210997
unique,1145040,1256,363,1105872,18787,828087,1208460,42772,1207408
top,"[{'S': 1, 'AuId': 2617818522, 'AfN': 'beckman ...","{'CN': 'cleo', 'CId': 2623670637}",2018-01-01,"[{'FId': 554144382, 'FN': 'performance art'}, ...","{'JN': 'plos one', 'JId': 202381698}","[2028520323, 1657433554, 2782514705, 273981532...",open access is there a predator at the door,PLOS ONE,"['access', 'door', 'open', 'predator']"
freq,3810,1643,182147,4454,14841,1206,11,14841,11


In [115]:
df[df.duplicated(["Title"])].sort_values(by=["Title"])

,Row_num,Authors,Conference,Citation_count,Date,ECC,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity,month


In [116]:
# df 
df.drop_duplicates(subset=['Title'], inplace=True)

In [125]:
df.isna().sum()

Authors                 0
Conference        1157829
Citation_count          0
Date                    0
ECC                     0
Domain              13793
Id                      0
Journal            191087
References         373689
Title                   0
Venue               63804
Words                   3
Year                    0
Popularity              0
month                   0
dtype: int64

In [95]:
df["Conference"].value_counts(dropna=False)

NaN                                                      1160211
{'CN': 'cleo', 'CId': 2623670637}                           1643
{'CN': 'chi', 'CId': 1163450153}                            1472
{'CN': 'icra', 'CId': 1163902177}                           1259
{'CN': 'icassp', 'CId': 1121227772}                         1139
{'CN': 'icc', 'CId': 1130451194}                            1027
{'CN': 'cvpr', 'CId': 1158167855}                           1004
{'CN': 'aaai', 'CId': 1184914352}                            873
{'CN': 'nips', 'CId': 1127325140}                            846
{'CN': 'acc', 'CId': 1190039108}                             794
{'CN': 'ijcai', 'CId': 1203999783}                           628
{'CN': 'icml', 'CId': 1180662882}                            622
{'CN': 'iscas', 'CId': 1192710900}                           544
{'CN': 'icse', 'CId': 1174403976}                            441
{'CN': 'ccc', 'CId': 2623792093}                             414
{'CN': 'interspeech', 'CI

In [117]:
df.iloc[0:1,-1]

0    1
Name: month, dtype: int64

In [81]:
df["kesEntityId"].is_unique

True

In [85]:
df["Id"].is_unique

True

In [91]:
# expected citation count is almost same as citation count
df[df["Citation_count"] != (df["ECC"])]

,Row_num,Authors,Conference,Citation_count,Date,ECC,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity
134,134,"[{'AuId': 2778608201, 'S': 1, 'AuN': 'f arenou...",NaN,33,2018-08-01,50,"[{'FId': 202799725, 'FN': 'accuracy and precis...",2799268730,"{'JN': 'astron astrophys', 'JId': 205231332}","[2111817932, 2142220522, 2798336535, 213372565...",gaia data release 2 catalogue validation,Astronomy and Astrophysics,"['2', 'catalogue', 'data', 'gaia', 'release', ...",2018,10136560,-18.243
228,228,"[{'S': 1, 'AuId': 2070030126, 'AfN': 'columbia...",NaN,48,2018-01-25,50,"[{'FId': 2778247410, 'FN': 'solanezumab'}, {'F...",2787612390,"{'JN': 'n engl j med', 'JId': 62468778}","[1847168837, 2156220037, 2063808031, 232933607...",trial of solanezumab for mild dementia due to ...,The New England Journal of Medicine,"['alzheimer', 'dementia', 'disease', 'mild', '...",2018,12076216,-18.422
298,298,"[{'S': 1, 'AuId': 2153068280, 'AfN': 'universi...",NaN,34,2018-01-01,50,"[{'FId': 189206191, 'FN': 'genomics'}, {'FId':...",2609984900,"{'JN': 'amer j psychiat', 'JId': 116025658}","[2134783591, 2113784410, 2098597355, 203315153...",psychiatric genomics an update and an agenda,American Journal of Psychiatry,"['agenda', 'genomics', 'psychiatric', 'update']",2018,13182132,-18.511
385,385,"[{'S': 1, 'AuId': 2104561707, 'AfN': 'arthriti...",NaN,36,2018-06-01,50,"[{'FId': 137886200, 'FN': 'biopsychosocial mod...",2791379829,"{'JN': 'lancet', 'JId': 49861241}","[2296986887, 2157244868, 2160769294, 258841006...",prevention and treatment of low back pain evid...,The Lancet,"['back', 'challenges', 'directions', 'evidence...",2018,14520434,-18.610
423,423,"[{'S': 1, 'AuId': 2273142510, 'AfN': 'liverpoo...",NaN,33,2018-09-14,50,"[{'FId': 125857072, 'FN': 'star formation'}, {...",2775727542,"{'JN': 'annu rev astron astrophys', 'JId': 298...",[2773772132],multiple stellar populations in globular clusters,Annual Review of Astronomy and Astrophysics,"['clusters', 'globular', 'multiple', 'populati...",2018,14932355,-18.640
1088,1088,"[{'AuId': 1978048194, 'S': 1, 'AuN': 'jessica ...",NaN,33,2018-03-07,50,"[{'FId': 63830546, 'FN': 'scientific method'},...",2791681873,"{'JN': 'nature', 'JId': 137773608}","[2156098321, 2126930838, 2157823046, 200044254...",meta analysis and the science of research synt...,Nature,"['meta', 'research', 'science', 'synthesis']",2018,19486089,-18.916
1655,1655,"[{'AuId': 2133441925, 'S': 1, 'AuN': 'steven r...",NaN,35,2018-02-01,50,"[{'FId': 2778715236, 'FN': 'discontinuation'},...",2766957057,"{'JN': 'jbmr', 'JId': 128199769}","[2044178924, 2160005424, 2482202957, 216305816...",vertebral fractures after discontinuation of d...,Journal of Bone and Mineral Research,"['controlled', 'denosumab', 'discontinuation',...",2018,21549747,-19.020
1966,1966,"[{'S': 1, 'AuId': 2319516946, 'AfN': 'delft un...",NaN,35,2018-04-06,50,"[{'FId': 22322919, 'FN': 'genomic organization...",2788376770,"{'JN': 'science', 'JId': 3880285}","[2073504534, 1995311028, 1964441323, 212108365...",real time imaging of dna loop extrusion by con...,Science,"['condensin', 'dna', 'extrusion', 'imaging', '...",2018,22391951,-19.061
2372,2372,"[{'S': 1, 'AuId': 2147956303, 'AfN': 'universi...",NaN,48,2018-05-01,50,"[{'FId': 73949472, 'FN': 'multijunction photov...",2800899574,"{'JN': 'nature energy', 'JId': 2764528046}","[2096320304, 2071134066, 2030106935, 261520321...",high fabrication yield organic tandem photovol...,Nature Energy,"['15', 'combining', 'efficiency', 'fabrication...",2018,23762922,-19.113
2458,2458,"[{'S': 1, 'AuId': 2177864800, 'AfN': 'translat...",NaN,34,2018-02-01,50,"[{'FId': 2779829264, 'FN': 'dulaglutide'}, {'F...",2787255324,"{'JN': 'lancet diabetes endocrinol', 'JId': 25...","[2059158127, 1526927233, 2564042923, 251951039...",semaglutide versus dulaglutide once weekly in ...,The Lancet Diabetes & Endocrinology,"['2', '3b', '7', 'diabetes', 'dulaglutide', 'l...",2018,23987300,-19.124


In [129]:
# create month field
df["Month"] = pd.to_datetime(df["Date"]).dt.month

In [130]:
df['Month'].head(5)

0    1
1    2
2    3
3    4
4    8
Name: Month, dtype: int64

In [123]:
def clean_data(df):
    #removing kesEntityId as simillar fields are already present
    df.drop(columns=["Row_num","kesEntityId"],inplace=True)
    df["Popularity"] = df["Popularity"].apply(pd.to_numeric,downcast='float')
    df["Year"] = df["Year"].apply(pd.to_numeric,downcast='unsigned')
    df["Month"] = df["Month"].apply(pd.to_numeric,downcast='unsigned')
    

In [21]:
def clean_data(df)
    #since first column contains row number
    df.drop(df.columns[0], axis=1, inplace=True)
    #removing  and kesEntityId as simillar fields are already present
    df.drop(columns=["Row_num","ECC","kesEntityId"])
    df["Popularity"] = df["Popularity"].apply(pd.to_numeric,downcast='float')
    df["Year"] = df["Year"].apply(pd.to_numeric,downcast='unsigned')
    df = df.dropna(subset=['Venue'])

,AA,C,CC,D,ECC,F,Id,J,RId,Ti,VFN,W,Y,kesEntityId,logprob
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064
1,"[{'S': 1, 'AuId': 2618360078, 'AfN': 'technica...",NaN,1787,2018-02-23,1787,"[{'FId': 2779853757, 'FN': 'penicillic acid'},...",2789134836,"{'JN': 'world mycotoxin journal', 'JId': 70096...","[2073569767, 1989318771, 2109782899, 214495155...",a critical review of producers of small lacton...,World Mycotoxin Journal,"['acid', 'critical', 'lactone', 'moniliformin'...",2018,1349261,-16.192
2,"[{'AuId': 2498139363, 'S': 1, 'AuN': 'r r m pa...",NaN,1785,2018-03-01,1785,"[{'FId': 2779550045, 'FN': 'patulin'}, {'FId':...",2786183769,"{'JN': 'int j food microbiol', 'JId': 132626406}",NaN,alternative patulin pathway unproven,International Journal of Food Microbiology,"['alternative', 'pathway', 'patulin', 'unproven']",2018,1350611,-16.193
3,"[{'S': 1, 'AuId': 2127898042, 'AfN': 'google',...",NaN,1957,2018-04-01,1957,"[{'FId': 65885262, 'FN': 'scale space segmenta...",2412782625,"{'JN': 'pami', 'JId': 199944782}","[1686810756, 2618530766, 2194775991, 209711776...",deeplab semantic image segmentation with deep ...,IEEE Transactions on Pattern Analysis and Mach...,"['atrous', 'connected', 'convolution', 'convol...",2018,1635532,-16.388
4,"[{'S': 1, 'AuId': 2761591088, 'AfN': 'leiden u...",NaN,293,2018-08-01,293,"[{'FId': 2778316820, 'FN': 'photometric system...",2798336535,"{'JN': 'astron astrophys', 'JId': 205231332}","[2011301426, 2142220522, 2061939373, 205293750...",gaia data release 2 summary of the contents an...,Astronomy and Astrophysics,"['2', 'contents', 'data', 'gaia', 'properties'...",2018,1744805,-16.455
5,"[{'S': 1, 'AuId': 146194825, 'AfN': 'universit...",NaN,840,2018-02-13,840,"[{'FId': 2779158730, 'FN': 'fecal occult blood...",2433575101,"{'JN': 'jama', 'JId': 172573765}","[2116542352, 2132528779, 2145150141, 213479352...",screening for colorectal cancer us preventive ...,JAMA,"['cancer', 'colorectal', 'force', 'preventive'...",2018,1761273,-16.464
6,"[{'AuId': 2162505138, 'S': 1, 'AuN': 'rebecca ...",NaN,1324,2018-01-01,1324,"[{'FId': 179755657, 'FN': 'mortality rate'}, {...",2781525129,"{'JN': 'ca a cancer j clin', 'JId': 126094547}","[130099911, 2134793522, 2089300758, 2042472927...",cancer statistics 2018,CA: A Cancer Journal for Clinicians,"['2018', 'cancer', 'statistics']",2018,1893460,-16.539
7,"[{'S': 1, 'AuId': 2895889280, 'AfN': 'american...",NaN,618,2018-10-23,618,"[{'FId': 84393581, 'FN': 'blood pressure'}, {'...",2768267412,"{'JN': 'j amer coll cardiol', 'JId': 60865174}","[2156121263, 2133416234, 2019071303, 211005231...",2017 acc aha aapa abc acpm ags apha ash aspc n...,Journal of the American College of Cardiology,"['2017', 'aapa', 'abc', 'acc', 'acpm', 'adults...",2018,1924596,-16.556
8,"[{'AuId': 2285399872, 'S': 1, 'AuN': 'jeanmich...",NaN,827,2018-08-01,827,"[{'FId': 2778867932, 'FN': 'paritaprevir'}, {'...",2185218296,"{'JN': 'j hepatol', 'JId': 83651209}","[2027545702, 2127685726, 2143559032, 212596379...",easl recommendations on treatment of hepatitis c,Journal of Hepatology,"['c', 'easl', 'hepatitis', 'recommendations', ...",2018,2141891,-16.665
9,"[{'S': 1, 'AuId': 2309940897, 'AfN': 'johns ho...",NaN,820,2018-02-01,820,"[{'FId': 33070731, 'FN': 'toxicology'}, {'FId'...",2779588417,"{'JN': 'toxicol sci', 'JId': 186282544}","[2319741579, 2605278466, 2685494954, 273903073...",green toxicology know early about and avoid to...,Toxicological Sciences,"['avoid', 'early', 'green', 'know', 'liabiliti...",2018,2240387,-16.711


In [ ]:
clean_data(df)

In [3]:
df_pub = pd.read_csv("Data/ProcessedScopusData.csv")

In [4]:
def append_pub_score(df):
    print("Unique Publications from Scopus:",len(df_pub.Title.unique()))
    # Analysing Common Publications
    vfn_list = df.Venue.unique().tolist()
    scopus_list = df_pub.Title.unique().tolist()
    count = 0
    for pub in scopus_list:
        if pub in vfn_list:
            count +=1
    print("Publications present in Dataset:",count)
    print("Shape Before",df.shape)
    df = df[df.Venue.isin(scopus_list)]
    print("Shape After",df.shape)
    #score appending
    score_dict = pd.Series(df_pub.SJR.values,index=df_pub.Title).to_dict()
    df["Publication_Rank"] = df.apply(lambda row: score_dict[row["VFN"]],axis = 1)
    return df

def extract_field(row):
    val = row["F"]
    index = val.rfind("FN")
    val = val[index+3:len(val)]
    val = val.strip(":}] '")
    return val

In [5]:
df = append_pub_score(df)

NameError: name 'df' is not defined

In [6]:
label_dict = {"C":0,"J":1,"CJ":2,"O":3}
def find_conference_type(row):
    if pd.isnull(row["Conference"]) and pd.isnull(row["Journal"]):
        return label_dict[O]
    elif pd.isnull(row["Conference"]):
        return label_dict["J"]
    elif pd.isnull(row["Journal"]):
        return label_dict["C"]
    else:
        return label_dict["CJ"]

In [7]:
df["Publication Type"] = df.apply(lambda row:find_conference_type(row),axis=1)

NameError: name 'df' is not defined